In [104]:
from typing import Annotated
import json
import re

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


class State(TypedDict):
    messages: Annotated[list, add_messages]


# Define the task schema for the course manager
class Task(BaseModel):
    task_1: str = Field(description="Task for the team 1")
    detail_1: str = Field(description="description of the task_1")
    task_2: str = Field(description="Task for the team 2")
    detail_2: str = Field(description="description of the task_2")
    task_3: str = Field(description="Task for the team 3")
    detail_3: str = Field(description="description of the task_3")
    task_4: str = Field(description="Task for the team 4")
    detail_4: str = Field(description="description of the task_4")

# Define the module schema for team outputs
class TeamModules(BaseModel):
    module_1: str = Field(description="title of the module 1")
    module_2: str = Field(description="title of the module 2")
    module_3: str = Field(description="title of the module 3")

# Initialize model
model = ChatOllama(model="llama3.1:8b")

# Initialize output parsers
manager_parser = JsonOutputParser(pydantic_object=Task)
team1_parser = JsonOutputParser(pydantic_object=TeamModules)
team2_parser = JsonOutputParser(pydantic_object=TeamModules)
team3_parser = JsonOutputParser(pydantic_object=TeamModules)
team4_parser = JsonOutputParser(pydantic_object=TeamModules)

# Create the manager prompt template
manager_template = """
You are an expert course designer team manager, You have PhD in dividing tasks among teams.

User requested course on: {Course}

You are managing 4 teams under you. Your task is to divide the course creation task into 4 different tasks and assign them to team leaders.

Please create the tasks in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
"""

manager_prompt = ChatPromptTemplate.from_template(manager_template)
manager_prompt = manager_prompt.partial(format_instructions=manager_parser.get_format_instructions())

# Create the team1 prompt template
team1_template = """
You are an expert team for the course creator team, your job is to take task from the manager and divide it to small manageable 3 modules.

Here is the task received from the manager side:
Task: {task_1}
Description: {detail_1}

Strictly follow following instructions:
Please create the tasks in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
every key value of the json should be in the double quots only.
"""

team1_prompt = ChatPromptTemplate.from_template(team1_template)
team1_prompt = team1_prompt.partial(format_instructions=team1_parser.get_format_instructions())

team2_template = """
You are an expert team for the course creator team, your job is to take task from the manager and divide it to small manageable 3 modules.

Here is the task received from the manager side:
Task: {task_2}
Description: {detail_2}

Strictly follow following instructions:
Please create the tasks in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
every key value of the json should be in the double quots only.
"""

team2_prompt = ChatPromptTemplate.from_template(team2_template)
team2_prompt = team2_prompt.partial(format_instructions=team2_parser.get_format_instructions())

# Define the course manager node function
def course_manager(state: State):
    try:
        # Get the course content from the user's first message
        course_content = state["messages"][0].content
        
        # Generate the prompt with the course content
        formatted_prompt = manager_prompt.format_messages(Course=course_content)
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # We'll just pass the raw content directly
        return {"messages": [{"role": "assistant", "content": model_response.content}]}
    except Exception as e:
        # Return an error message if processing fails
        return {"messages": [{"role": "assistant", "content": f"Error in course manager: {str(e)}. Please try again."}]}

# Define the team1 node function
def team1_leader(state: State):
    try:
        # Get the task from the manager's response
        print("ssssssssssssssttttttttttaaaaaaaaa", state)
        manager_message = state["messages"][-1].content
        
        # Extract JSON from the message - handling potential formatting issues
        try:
            # First try direct JSON parsing
            json_data = json.loads(manager_message)
        except json.JSONDecodeError:
            # If that fails, try to extract JSON using regex
            json_match = re.search(r'\{.*\}', manager_message, re.DOTALL)
            if not json_match:
                raise ValueError("Could not find JSON in manager response")
            json_data = json.loads(json_match.group(0))
        print("jjjjjjjjjjjaaaaaaaasssssssssooooooooooooonnnnnnnnnnnnn", json_data)
        # Extract the task info
        properties = json_data.get("properties", None)
        if properties:
            json_data = properties
        team1_task = json_data.get("task_1")
        team1_detail = json_data.get("detail_1")
        
        if not team1_task or not team1_detail:
            raise ValueError("Missing task_1 or detail_1 in manager response")
            
        # Generate the prompt with the team task
        formatted_prompt = team1_prompt.format_messages(
            task_1=team1_task, 
            detail_1=team1_detail
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Return the raw response
        return {"messages": [{"role": "assistant", "content": model_response.content}]}
    except Exception as e:
        # Return an error message if processing fails
        return {"messages": [{"role": "assistant", "content": f"Error in team1 processing: {str(e)}. Please try again."}]}

# Define the team1 node function
def team2_leader(state: State):
    try:
        # Get the task from the manager's response
        print("ssssssssssssssttttttttttaaaaaaaaa", state)
        manager_message = state["messages"][1].content
        
        # Extract JSON from the message - handling potential formatting issues
        try:
            # First try direct JSON parsing
            json_data = json.loads(manager_message)
        except json.JSONDecodeError:
            # If that fails, try to extract JSON using regex
            json_match = re.search(r'\{.*\}', manager_message, re.DOTALL)
            if not json_match:
                raise ValueError("Could not find JSON in manager response")
            json_data = json.loads(json_match.group(0))
        print("jjjjjjjjjjjaaaaaaaasssssssssooooooooooooonnnnnnnnnnnnn", json_data)
        # Extract the task info
        properties = json_data.get("properties", None)
        if properties:
            json_data = properties
        team2_task = json_data.get("task_2")
        team2_detail = json_data.get("detail_2")
        
        if not team2_task or not team2_detail:
            raise ValueError("Missing task_2 or detail_2 in manager response")
            
        # Generate the prompt with the team task
        formatted_prompt = team2_prompt.format_messages(
            task_2=team2_task, 
            detail_2=team2_detail
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Return the raw response
        return {"messages": [{"role": "assistant", "content": model_response.content}]}
    except Exception as e:
        # Return an error message if processing fails
        return {"messages": [{"role": "assistant", "content": f"Error in team1 processing: {str(e)}. Please try again."}]}


# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("course_manager", course_manager)
graph_builder.add_node("team1_leader", team1_leader)
graph_builder.add_node("team2_leader", team2_leader)

# Define the edges
graph_builder.add_edge(START, "course_manager")
graph_builder.add_edge("course_manager", "team1_leader")
graph_builder.add_edge("course_manager", "team2_leader")
graph_builder.add_edge("team1_leader", END)

# Compile the graph
graph = graph_builder.compile()

# Function to run the graph
def process_course_request(course_topic):
    # Create proper message format
    initial_state = {
        "messages": [
            {
                "role": "user", 
                "content": course_topic,
                "type": "human"
            }
        ]
    }
    result = graph.invoke(initial_state)
    return result["messages"][-1]

# Example usage
if __name__ == "__main__":
    result = process_course_request("Python Programming for Beginners")
    print(result)

ssssssssssssssttttttttttaaaaaaaaa {'messages': [HumanMessage(content='Python Programming for Beginners', additional_kwargs={'type': 'human'}, response_metadata={}, id='d7179829-a5fc-4bd0-a644-577d9669e115'), AIMessage(content='```json\n{\n  "task_1": "Develop Course Outline and Syllabus",\n  "detail_1": "Create a detailed outline of the course covering all topics, including Python basics, data structures, file input/output, exceptions, and more. Develop a syllabus that includes assignments, quizzes, and projects.",\n  \n  "task_2": "Design Interactive Code Exercises",\n  "detail_2": "Develop interactive code exercises for each topic covered in the course. Use platforms like Repl.it or Google Colab to make it easy for learners to write and run Python code.",\n  \n  "task_3": "Create Engaging Video Lessons",\n  "detail_3": "Produce high-quality video lessons that explain complex concepts in an engaging way. Use animations, diagrams, and real-world examples to illustrate key points.",\n  

In [114]:
from IPython.display import Image, display
graph_builder.get_graph().print_ascii()
# display(Image(graph.get_graph().print_ascii()))


AttributeError: 'StateGraph' object has no attribute 'get_graph'

In [115]:
"""
Fixed Course Generation System using LangGraph and Llama 3.1
This version addresses concurrent update issues by using a more sequential flow
"""

from typing import Annotated, List, Dict, Any
import json
import re
import os
from datetime import datetime
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


class State(TypedDict):
    """State type for the course generation graph"""
    messages: Annotated[list, add_messages]
    course_topic: str
    manager_output: Dict
    team1_output: Dict
    team2_output: Dict
    team3_output: Dict
    team4_output: Dict
    module_contents: Dict


# Define the task schema for the course manager
class Task(BaseModel):
    """Task schema for the course manager"""
    task_1: str = Field(description="Task for the team 1 - Course structure")
    detail_1: str = Field(description="Description of the task_1")
    task_2: str = Field(description="Task for the team 2 - Core content")
    detail_2: str = Field(description="Description of the task_2")
    task_3: str = Field(description="Task for the team 3 - Exercises and assessments")
    detail_3: str = Field(description="Description of the task_3")
    task_4: str = Field(description="Task for the team 4 - Resources and examples")
    detail_4: str = Field(description="Description of the task_4")


# Define the module schema for team outputs
class TeamModules(BaseModel):
    """Module schema for team outputs"""
    module_1: str = Field(description="Title of the module 1")
    description_1: str = Field(description="Description of module 1")
    module_2: str = Field(description="Title of the module 2")
    description_2: str = Field(description="Description of module 2")
    module_3: str = Field(description="Title of the module 3")
    description_3: str = Field(description="Description of module 3")


# Define the content schema for module content
class ModuleContent(BaseModel):
    """Content schema for module details"""
    title: str = Field(description="Title of the module")
    introduction: str = Field(description="Introduction to the module")
    key_points: List[str] = Field(description="Key points to learn")
    content: str = Field(description="Main content of the module")
    examples: List[str] = Field(description="Examples that illustrate concepts")
    summary: str = Field(description="Summary of the module")


# Initialize LLM
model = ChatOllama(model="llama3.1:8b")

# Initialize output parsers
manager_parser = JsonOutputParser(pydantic_object=Task)
team_parser = JsonOutputParser(pydantic_object=TeamModules)
content_parser = JsonOutputParser(pydantic_object=ModuleContent)


# Create prompt templates
manager_template = """
You are an expert course designer team manager with extensive experience in education.

User requested a course on: {course_topic}

You are managing 4 teams:
1. Course Structure Team: Responsible for overall organization and learning path
2. Core Content Team: Responsible for developing main educational content
3. Assessment Team: Responsible for exercises, quizzes, and practical applications
4. Resources Team: Responsible for supplementary materials and examples

Divide the course creation tasks for this topic into 4 distinct areas for each team.

Please create the tasks in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
"""

team_template = """
You are an expert course content developer specializing in {team_specialty}.

Here is the task received from the manager:
Task: {team_task}
Description: {team_detail}

Your job is to propose three well-structured modules that fulfill this task for the course on {course_topic}.
Each module should have a clear title and description.

Please create the modules in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
"""

content_template = """
You are an expert educational content creator specializing in creating clear, engaging learning materials.

You're creating detailed content for the following module in a course about {course_topic}:

Module Title: {module_title}
Module Description: {module_description}

Create comprehensive, expert-level content for this module including:
- A thorough introduction
- List of key points that should be learned
- Detailed main content with clear explanations
- Practical examples that illustrate concepts
- A concise summary that reinforces learning

Please create the module content in a JSON format with the following structure:
{format_instructions}

Return ONLY the JSON without any additional text or explanation.
"""

# Set up prompt templates with parsers
manager_prompt = ChatPromptTemplate.from_template(manager_template)
manager_prompt = manager_prompt.partial(format_instructions=manager_parser.get_format_instructions())

team_prompt = ChatPromptTemplate.from_template(team_template)
team_prompt = team_prompt.partial(format_instructions=team_parser.get_format_instructions())

content_prompt = ChatPromptTemplate.from_template(content_template)
content_prompt = content_prompt.partial(format_instructions=content_parser.get_format_instructions())


# Helper function for JSON extraction
def parse_json(text):
    """Helper function to extract and parse JSON from text"""
    try:
        # First try direct JSON parsing
        return json.loads(text)
    except json.JSONDecodeError:
        # If that fails, try to extract JSON using regex
        json_match = re.search(r'\{.*\}', text, re.DOTALL)
        if not json_match:
            raise ValueError("Could not find JSON in response")
        return json.loads(json_match.group(0))


# Define node functions
def course_manager(state: State) -> State:
    """Course manager that divides the work among teams"""
    try:
        # Get the course topic
        course_topic = state["course_topic"]
        
        # Generate the prompt with the course content
        formatted_prompt = manager_prompt.format_messages(course_topic=course_topic)
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Parse the JSON response
        manager_output = parse_json(model_response.content)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Course manager has divided the work into specialized tasks."}],
            "course_topic": course_topic,
            "manager_output": manager_output,
            "team1_output": {},
            "team2_output": {},
            "team3_output": {},
            "team4_output": {},
            "module_contents": {}
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in course manager: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": {},
            "team1_output": {},
            "team2_output": {},
            "team3_output": {},
            "team4_output": {},
            "module_contents": {}
        }


def team1_leader(state: State) -> State:
    """Team 1 leader - Course structure and learning path design"""
    try:
        # Get relevant task information
        manager_output = state["manager_output"]
        course_topic = state["course_topic"]
        
        team_task = manager_output.get("task_1")
        team_detail = manager_output.get("detail_1")
        
        if not team_task or not team_detail:
            raise ValueError(f"Missing task_1 or detail_1 in manager response")
        
        # Generate the prompt with the team task
        formatted_prompt = team_prompt.format_messages(
            team_specialty="course structure and learning path design",
            team_task=team_task,
            team_detail=team_detail,
            course_topic=course_topic
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Parse the JSON response
        team_output = parse_json(model_response.content)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Team 1 has created module outlines for course structure."}],
            "course_topic": course_topic,
            "manager_output": manager_output,
            "team1_output": team_output,
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in team 1 processing: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": {},
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }


def team2_leader(state: State) -> State:
    """Team 2 leader - Core educational content development"""
    try:
        # Get relevant task information
        manager_output = state["manager_output"]
        course_topic = state["course_topic"]
        
        team_task = manager_output.get("task_2")
        team_detail = manager_output.get("detail_2")
        
        if not team_task or not team_detail:
            raise ValueError(f"Missing task_2 or detail_2 in manager response")
        
        # Generate the prompt with the team task
        formatted_prompt = team_prompt.format_messages(
            team_specialty="core educational content development",
            team_task=team_task,
            team_detail=team_detail,
            course_topic=course_topic
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Parse the JSON response
        team_output = parse_json(model_response.content)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Team 2 has created module outlines for core content."}],
            "course_topic": course_topic,
            "manager_output": manager_output,
            "team1_output": state["team1_output"],
            "team2_output": team_output,
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in team 2 processing: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": {},
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }


def team3_leader(state: State) -> State:
    """Team 3 leader - Assessments and practical exercises"""
    try:
        # Get relevant task information
        manager_output = state["manager_output"]
        course_topic = state["course_topic"]
        
        team_task = manager_output.get("task_3")
        team_detail = manager_output.get("detail_3")
        
        if not team_task or not team_detail:
            raise ValueError(f"Missing task_3 or detail_3 in manager response")
        
        # Generate the prompt with the team task
        formatted_prompt = team_prompt.format_messages(
            team_specialty="assessments and practical exercises",
            team_task=team_task,
            team_detail=team_detail,
            course_topic=course_topic
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Parse the JSON response
        team_output = parse_json(model_response.content)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Team 3 has created module outlines for assessments."}],
            "course_topic": course_topic,
            "manager_output": manager_output,
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": team_output,
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in team 3 processing: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": {},
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }


def team4_leader(state: State) -> State:
    """Team 4 leader - Supplementary resources and examples"""
    try:
        # Get relevant task information
        manager_output = state["manager_output"]
        course_topic = state["course_topic"]
        
        team_task = manager_output.get("task_4")
        team_detail = manager_output.get("detail_4")
        
        if not team_task or not team_detail:
            raise ValueError(f"Missing task_4 or detail_4 in manager response")
        
        # Generate the prompt with the team task
        formatted_prompt = team_prompt.format_messages(
            team_specialty="supplementary resources and examples",
            team_task=team_task,
            team_detail=team_detail,
            course_topic=course_topic
        )
        
        # Get the model response
        model_response = model.invoke(formatted_prompt)
        
        # Parse the JSON response
        team_output = parse_json(model_response.content)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": "Team 4 has created module outlines for resources."}],
            "course_topic": course_topic,
            "manager_output": manager_output,
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": team_output,
            "module_contents": state["module_contents"]
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in team 4 processing: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": {},
            "module_contents": state["module_contents"]
        }


def create_module_content(state: State) -> State:
    """Create detailed content for the first module of each team"""
    try:
        # Get relevant information
        course_topic = state["course_topic"]
        module_contents = state["module_contents"].copy()
        
        # Process one module from each team
        teams = [
            ("team1", state["team1_output"]),
            ("team2", state["team2_output"]),
            ("team3", state["team3_output"]),
            ("team4", state["team4_output"])
        ]
        
        module_titles = []
        
        for team_name, team_output in teams:
            if team_output and "module_1" in team_output and "description_1" in team_output:
                module_title = team_output["module_1"]
                module_description = team_output["description_1"]
                
                # Generate the prompt for content creation
                formatted_prompt = content_prompt.format_messages(
                    course_topic=course_topic,
                    module_title=module_title,
                    module_description=module_description
                )
                
                # Get the model response
                model_response = model.invoke(formatted_prompt)
                
                # Parse the JSON response
                content_output = parse_json(model_response.content)
                
                # Add to module contents
                module_key = f"{team_name}_module1"
                module_contents[module_key] = content_output
                module_titles.append(module_title)
        
        module_list = ", ".join(module_titles)
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Created detailed content for modules: {module_list}"}],
            "course_topic": course_topic,
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": module_contents
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in content creation: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }


def course_summary(state: State) -> State:
    """Create a final course summary integrating all components"""
    try:
        course_topic = state["course_topic"]
        team1_output = state["team1_output"]
        team2_output = state["team2_output"]
        team3_output = state["team3_output"] 
        team4_output = state["team4_output"]
        module_contents = state["module_contents"]
        
        # Count modules created
        print("***********************************", state)
        module_count = len(module_contents)
        
        # Get all module titles
        modules = []
        
        for team_data in [team1_output, team2_output, team3_output, team4_output]:
            for i in range(1, 4):  # Modules 1-3
                module_title = team_data.get(f"module_{i}")
                if module_title:
                    modules.append(module_title)
        
        # Build a course summary
        summary = f"""
# Course Generation Complete: Expert Course on {course_topic}

## Course Overview
An expert-level comprehensive course on {course_topic}, designed with a structured learning path
and detailed content across multiple specialized areas.

## Course Structure
The course contains {len(modules)} modules across 4 specialized areas:
{', '.join(modules[:5])}{'...' if len(modules) > 5 else ''}

## Content Statistics
- Total modules outlined: {len(modules)}
- Modules with detailed content: {module_count}

## Module Breakdown
- Course Structure: {', '.join([team1_output.get(f'module_{i}', '') for i in range(1, 4) if team1_output.get(f'module_{i}', '')])}
- Core Content: {', '.join([team2_output.get(f'module_{i}', '') for i in range(1, 4) if team2_output.get(f'module_{i}', '')])}
- Assessments: {', '.join([team3_output.get(f'module_{i}', '') for i in range(1, 4) if team3_output.get(f'module_{i}', '')])}
- Resources: {', '.join([team4_output.get(f'module_{i}', '') for i in range(1, 4) if team4_output.get(f'module_{i}', '')])}

## Next Steps
All course materials have been generated. You can access the full course content
in the returned data structure.
"""
        
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": summary}],
            "course_topic": course_topic,
            "manager_output": state["manager_output"],
            "team1_output": team1_output,
            "team2_output": team2_output,
            "team3_output": team3_output,
            "team4_output": team4_output,
            "module_contents": module_contents
        }
    except Exception as e:
        return {
            "messages": state["messages"] + [{"role": "assistant", "content": f"Error in course summary: {str(e)}. Please try again."}],
            "course_topic": state["course_topic"],
            "manager_output": state["manager_output"],
            "team1_output": state["team1_output"],
            "team2_output": state["team2_output"],
            "team3_output": state["team3_output"],
            "team4_output": state["team4_output"],
            "module_contents": state["module_contents"]
        }


# Build the course generation graph
def build_course_generation_graph():
    """Build and return the course generation workflow graph"""
    graph_builder = StateGraph(State)
    
    # Add all nodes
    graph_builder.add_node("course_manager", course_manager)
    graph_builder.add_node("team1_leader", team1_leader)
    graph_builder.add_node("team2_leader", team2_leader)
    graph_builder.add_node("team3_leader", team3_leader)
    graph_builder.add_node("team4_leader", team4_leader)
    graph_builder.add_node("create_module_content", create_module_content)
    graph_builder.add_node("course_summary", course_summary)
    
    # Define sequential edges to avoid concurrent updates
    graph_builder.add_edge(START, "course_manager")
    graph_builder.add_edge("course_manager", "team1_leader")
    graph_builder.add_edge("team1_leader", "team2_leader")
    graph_builder.add_edge("team2_leader", "team3_leader")
    graph_builder.add_edge("team3_leader", "team4_leader")
    graph_builder.add_edge("team4_leader", "create_module_content")
    graph_builder.add_edge("create_module_content", "course_summary")
    graph_builder.add_edge("course_summary", END)
    
    # Compile the graph
    return graph_builder.compile()


# Function to run the graph
def generate_course(course_topic):
    """Generate a complete course on the specified topic"""
    # Initialize the graph
    graph = build_course_generation_graph()
    graph.get_graph().print_ascii()
    
    # Create initial state
    initial_state = {
        "messages": [{"role": "user", "content": f"Create an expert course on: {course_topic}"}],
        "course_topic": course_topic,
        "manager_output": {},
        "team1_output": {},
        "team2_output": {},
        "team3_output": {},
        "team4_output": {},
        "module_contents": {}
    }
    
    # Run the graph
    print(f"Generating course on: {course_topic}...")
    final_state = graph.invoke(initial_state)
    print("Course generation complete!")
    
    # Return the result
    return {
        "messages": final_state["messages"],
        "course_structure": {
            "team1": final_state["team1_output"],
            "team2": final_state["team2_output"],
            "team3": final_state["team3_output"],
            "team4": final_state["team4_output"]
        },
        "course_content": final_state["module_contents"]
    }


# Save course data to file
def save_course_to_file(course_data, filename=None):
    """Save generated course data to a JSON file"""
    if filename is None:
        # Create a filename based on the course topic
        course_topic = course_data.get("messages", [{}])[0].content
        topic_match = re.search(r'Create an expert course on: (.*)', course_topic)
        if topic_match:
            topic = topic_match.group(1)
        else:
            topic = "course"
            
        # Clean the topic for filename
        clean_topic = re.sub(r'[^\w\s-]', '', topic).strip().replace(' ', '_').lower()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{clean_topic}_{timestamp}.json"
    
    # Make sure the filename has .json extension
    if not filename.endswith('.json'):
        filename += '.json'
        
    try:
        with open(filename, "w") as f:
            json.dump(course_data, f, indent=2)
        print(f"Course data saved to {filename}")
        return filename
    except Exception as e:
        print(f"Error saving course data: {str(e)}")
        return None


# Example usage
if __name__ == "__main__":
    import sys
    
    # Get course topic from command line or use default
    course_topic = "Python Programming for Beginners"
    
    # Generate the course
    result = generate_course(course_topic)
    
    # Save to file
    save_course_to_file(result)
    
    # Print the last message (summary)
    print("\nCourse Summary:")
    print("=" * 50)
    print(result["messages"][-1].content)

      +-----------+        
      | __start__ |        
      +-----------+        
            *              
            *              
            *              
    +----------------+     
    | course_manager |     
    +----------------+     
            *              
            *              
            *              
    +--------------+       
    | team1_leader |       
    +--------------+       
            *              
            *              
            *              
    +--------------+       
    | team2_leader |       
    +--------------+       
            *              
            *              
            *              
    +--------------+       
    | team3_leader |       
    +--------------+       
            *              
            *              
            *              
    +--------------+       
    | team4_leader |       
    +--------------+       
            *              
            *              
            *       